In [2]:
#!pip install ipywidgets
#!pip install pivottablejs

import pandas as pd
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from erddapy import ERDDAP
import matplotlib.pyplot as plt
from matplotlib import style
import time
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import glob

from pivottablejs import pivot_ui


with open("erddap_servers.txt") as file_in:
    URLoptions = [""]
    for line in file_in:
        URLoptions.append(line)
        
dropdown_URLs = widgets.Dropdown(options = URLoptions)
#display(dropdown_URLs)
button_URLs = widgets.Button(description="Load URL for dataset list")
output = widgets.Output()
#display(button_URLs, output)
dropdown_Datasets = widgets.Dropdown(options = '')
#display(dropdown_Datasets)
button_DTS = widgets.Button(description="Load Dataset params")
#display(button_DTS)
#print("Y variable")
#dropdown_VarsY = widgets.Dropdown(options = '')
dropdown_VarsY=widgets.SelectMultiple(
    options=[''],
    description='Y variable',
    disabled=False
)
#display(dropdown_VarsY)
#print("X variable")
dropdown_VarsX = widgets.Dropdown(description="X variable",options = '')
#display(dropdown_VarsX)
button_Params = widgets.Button(description="Execute")
button_Map = widgets.Button(description="Plot map LON(Y variable)/LAT(X variable)",
    layout={'width': '300px'}
)

start=widgets.DatePicker(
    description='Start Date',
    disabled=False
)
end=widgets.DatePicker(
    description='End Date',
    disabled=False
)
#display(start)
#display(end)

okPNG=widgets.Checkbox(
    value=False,
    description='Save plot as image',
    disabled=False,
    indent=False
)
okxlsx=widgets.Checkbox(
    value=False,
    description='Save data as XLSX',
    disabled=False,
    indent=False
)
okcsv=widgets.Checkbox(
    value=False,
    description='Save data as CSV',
    disabled=False,
    indent=False
)
outputbox=widgets.Textarea(
    value='',
    placeholder='Ready....',
    description='Output Box:',
    disabled=False,
    layout={'width': '90%','height': '300px'}
)

DirPath = widgets.Text(
    description='Directory:',
    value='./')

button_ListDir = widgets.Button(description="List Directory")
filesInDir = widgets.Select(
    description='Dataset(s)',
    options='')

button_pivottablejs = widgets.Button(description="Use pivottablejs")

outputboxReport=widgets.Textarea(
    value='',
    placeholder='Ready....',
    description='Report Box:',
    disabled=False,
    layout={'width': '90%','height': '100px'}
)



DirPathCSV = widgets.Text(
    description='CSVs Dir:',
    value='./')
button_mergecsv = widgets.Button(description="Merge CSV")

DirPathXLS = widgets.Text(
    description='XLSs Dir:',
    value='./')
button_mergeXLS = widgets.Button(description="Merge XLS to CSV")

outputboxUtilities=widgets.Textarea(
    value='',
    placeholder='Ready....',
    description='Utilities Box:',
    disabled=False,
    layout={'width': '90%','height': '100px'}
)
#display(okPNG)
#display(okxlsx)
#display(okcsv)

#display(button_Params)
#display(button_Map)
#display(outputbox)
tab0 = VBox(children=[dropdown_URLs,
                        button_URLs, 
                        output,
                        dropdown_Datasets,
                        button_DTS,
                        dropdown_VarsY,
                        dropdown_VarsX,
                        start,
                        end,
                        okPNG,
                        okxlsx,
                        okcsv,
                        button_Params,
                        button_Map,
                        outputbox,
                      ])
tab1 = VBox(children=[DirPath,
                      button_ListDir,
                      filesInDir,
                      button_pivottablejs,
                      outputboxReport,
                      ])
tab2 = VBox(children=[DirPathCSV,
                      button_mergecsv,
                      DirPathXLS,
                      button_mergeXLS,
                      outputboxUtilities,
                      ])

tab = widgets.Tab(children=[tab0,tab1,tab2])
tab.set_title(0, 'SEARCH ERDDAP')
tab.set_title(1, 'Data Profiling')
tab.set_title(2, 'Utilities')
#VBox(children=[tab])
display(tab)
#display(DirPath)
#display(button_ListDir)
#display(filesInDir)
def mergeCSV(b):
    outputFileMerged = str(time.strftime("%Y%m%d%H%M%S")+'_merged_file.csv')        
    li = []
    optionsFilesCSV = glob.glob('{}/*'.format(DirPathCSV.value))
    for fileCSV in optionsFilesCSV:
        if fileCSV.endswith(".csv"):

            df = pd.read_csv(fileCSV, index_col=None, header=0)
            li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame.to_csv(outputFileMerged, index=False)
    outputboxUtilities.value=str("\n The files are merged in: "+outputFileMerged+" ")
    
def mergeXLS(b):
    outputFileMergedXLS = str(time.strftime("%Y%m%d%H%M%S")+'_merged_file_from_xls.csv')   
    # create a new dataframe to store the
    # merged excel file.
    excl_merged = pd.DataFrame()
    optionsFilesXLS = glob.glob('{}/*'.format(DirPathXLS.value))

    for fileXLS in optionsFilesXLS:
        if fileXLS.endswith(".xls") or fileXLS.endswith(".xlsx"):
            # appends the data into the excl_merged
            # dataframe.
            dataframeTMP = pd.read_excel(fileXLS)
            excl_merged = excl_merged.append(dataframeTMP)
    #outputboxUtilities.value=str(excl_merged)       
    excl_merged.to_csv(outputFileMergedXLS, index=False)
    outputboxUtilities.value=str("\n The files are merged in: "+outputFileMergedXLS+" ")      
            

def listDir_clicked(b):
    optionsFiles = glob.glob('{}/*'.format(DirPath.value))
    filesInDir.options = optionsFiles

def on_buttonURL_clicked(b):
    with output:
        clear_output(wait=False)
        #print("Button clicked.")
        print(dropdown_URLs.value)
        
        if dropdown_URLs.value!='':

            e = ERDDAP(server=str(dropdown_URLs.value))

            kw = {
                "min_time": "1900-01-01T00:00:00Z",
            }
            try:
                search_url = e.get_search_url(response="csv", **kw)
                search = pd.read_csv(search_url)
                DTSid = search["Dataset ID"].values
                DTSid_list = "\n".join(DTSid)
                msg_box = ''
                if len(DTSid)>50 :
                    print(str('WARNING Found '+str(len(DTSid))+' datasets: Are you sure you want to continue? Too many datasets can crash the software'))

                new_choices=[]
                new_choices.append('')
                for myid in DTSid:
                    new_choices.append(myid)
                    
                dropdown_Datasets.options = new_choices
                
            except Exception as e:
                print('WARNING! Exception ocurred')
                print(e)

def make_map():
    fig, ax = plt.subplots(
        figsize=(9, 9),
        subplot_kw=dict(projection=ccrs.PlateCarree())
    )
    ax.coastlines(resolution="10m")
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    return fig, ax
                
def on_buttonMap_clicked(b):
    myVar=dropdown_VarsY.value
    mySecondaryVar=dropdown_VarsX.value
    if myVar!='' and mySecondaryVar!='':
        e = ERDDAP(
            server= str(dropdown_URLs.value),
            protocol="tabledap",
            response="csv",
        )
        e.dataset_id = str(dropdown_Datasets.value)
        mylistVars = list(myVar)
        mylistVars.insert(0,mySecondaryVar)
            
        e.variables=mylistVars

        e.constraints = {
            "time>=": str(start.value)+"T00:00:00Z",
            "time<=": str(end.value)+"T23:59:59Z",}
        
        try:
            # Print the URL - check
            url = e.get_download_url()
            # Convert URL to pandas dataframe
            df_MySite = e.to_pandas(  
                parse_dates=True,
            ).dropna()
            # print the dataframe to check what data is in there specifically. 
            print(df_MySite.head())
            outputbox.value=str(df_MySite)
            #print(str(df_MySite.head()))
            print("Number of rows: "+str(len(df_MySite.index)))
            myrows=len(df_MySite.index)
            print("Number of rows pt2: "+str(myrows))
            # print the column names
            print (df_MySite.columns)
            fig = plt.figure()
            ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
            lon=df_MySite.iloc[:,1]
            lat=df_MySite.iloc[:,0]
            print(lon)
            print(lat)
            ax.scatter(lon, lat)
            ax.stock_img()
            ax.coastlines()
            ax.gridlines()
            plt.show()
        except Exception as e:
            print('WARNING! Exception ocurred')
            print(e)

            

    
def Reportpivottablejs(b):
    #with clear_output:
    FileToProfile=str(filesInDir.value)
    df= pd.read_csv(FileToProfile)
    pivot_ui(df)
    #report.show_html() 
    outputboxReport.value=str("\n The report has been generated, open in a browser the following file: pivottablejs.html ")
    print("The report has been generated, open in a browser the following file: pivottablejs.html ")
    
    

def on_buttonDTS_clicked(b):
    tmpDTS=str(dropdown_Datasets.value)
    if tmpDTS!='':
        print('tmpDTS')
        try:
            e = ERDDAP(server=str(dropdown_URLs.value))
            info_url = e.get_info_url(dataset_id=tmpDTS, response="csv")
            info = pd.read_csv(info_url)
            #print(info.head())
            print(info.to_string())

            rslt_df = info[info['Row Type'] == "variable"]
            
            dropdown_VarsY.options = rslt_df['Variable Name']
            
            dropdown_VarsX.options = rslt_df['Variable Name']

        except Exception as e:
            print('WARNING! Exception ocurred')
    else:
        print('Please, select a dataset')
        
def on_buttonVars_clicked(b):
    myVar=dropdown_VarsY.value
    mySecondaryVar=dropdown_VarsX.value
    print(myVar)
    for i in myVar:
        print(i)
    #if myVar!='' and mySecondaryVar!='':
    if myVar!='':
        e = ERDDAP(
            server= str(dropdown_URLs.value),
            protocol="tabledap",
            response="csv",
        )
        
        e.dataset_id = str(dropdown_Datasets.value)
        mylistVars = list(myVar)
        if okPNG.value==True:
            mylistVars.insert(0,mySecondaryVar)
            
        e.variables=mylistVars

        e.constraints = {
            "time>=": str(start.value)+"T00:00:00Z",
            "time<=": str(end.value)+"T23:59:59Z",}
        
        try:
            # Print the URL - check
            url = e.get_download_url()
            # Convert URL to pandas dataframe
            df_MySite = e.to_pandas(  
                parse_dates=True,
            ).dropna()
            # print the dataframe to check what data is in there specifically. 
            print(df_MySite.head())
            outputbox.value=str(df_MySite)
            #print(str(df_MySite.head()))
            print("Number of rows: "+str(len(df_MySite.index)))
            myrows=len(df_MySite.index)
            print("Number of rows pt2: "+str(myrows))
            # print the column names
            print (df_MySite.columns)
            
            '''
            plt.style.use("Solarize_Light2")
            ax = plt.gca()
            
            column_names = list(df_MySite.columns.values)
            print(column_names)
            myindex=0
            for i in column_names:
                if myindex==0:
                    print('No first field')
                else:
                    df_MySite.plot(kind='line',x=df_MySite.columns[0],y=df_MySite.columns[myindex],ax=ax, rot=90)
                myindex+=1

            plt.show()
            '''
            if okPNG.value==True:
                plt.style.use("Solarize_Light2")
                ax = plt.gca()
                column_names = list(df_MySite.columns.values)
                print(column_names)
                myindex=0
                for i in column_names:
                    if myindex==0:
                        print('No first field')
                    else:
                        df_MySite.plot(kind='line',x=df_MySite.columns[0],y=df_MySite.columns[myindex],ax=ax, rot=90)
                    myindex+=1

                #plt.show()
                savedPlot = str(time.strftime("%Y%m%d%H%M%S")+'_output.png')
                plt.savefig(savedPlot)
            
            if okxlsx.value==True:
                savedXlsx = str(time.strftime("%Y%m%d%H%M%S")+'_output.xlsx')
                df_MySite.to_excel(savedXlsx,sheet_name='Sheet_name_1')
                
            if okcsv.value==True:
                savedCSV = str(time.strftime("%Y%m%d%H%M%S")+'_output.csv')
                df_MySite.to_csv(savedCSV, encoding='utf-8')
                
        except Exception as e:
            print('WARNING! Exception ocurred')
            print(e)
    else:
        print("Please, select two params")
        
button_URLs.on_click(on_buttonURL_clicked)
button_DTS.on_click(on_buttonDTS_clicked)
button_Params.on_click(on_buttonVars_clicked)
button_Map.on_click(on_buttonMap_clicked)
button_ListDir.on_click(listDir_clicked)
button_pivottablejs.on_click(Reportpivottablejs)
button_mergecsv.on_click(mergeCSV)
button_mergeXLS.on_click(mergeXLS)